In [1]:
from tensorflow.keras.layers import Conv2D, BatchNormalization, Conv2DTranspose, MaxPooling2D, UpSampling2D, concatenate, Dropout
from tensorflow.keras.callbacks import EarlyStopping, History, ModelCheckpoint, TensorBoard, ReduceLROnPlateau
from tensorflow.keras.preprocessing.image import load_img
import matplotlib.pyplot as plt
from datetime import datetime
import tensorflow as tf
import numpy as np
import random
import cv2
import os

In [2]:
gpus = tf.config.list_physical_devices('GPU')
print("Los dispositivos encontrados son: ", gpus)
if gpus:
  try:
    for gpu in gpus:
      tf.config.experimental.set_memory_growth(gpu, True)
    logical_gpus = tf.config.experimental.list_logical_devices('GPU')
    print(len(gpus), "GPUs físicos,", len(logical_gpus), "GPUs lógicos")
  except RuntimeError as e:
    print(e)
print("----------------------------------------------------- \n")

Los dispositivos encontrados son:  [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]
1 GPUs físicos, 1 GPUs lógicos
----------------------------------------------------- 



In [3]:
#Dirección de directorios de entrenamient y validación
input_dir= "/home/revientaelp/Documentos/Bases de datos/Kika/300/ent/org"
target_dir= "/home/revientaelp/Documentos/Bases de datos/Kika/300/ent/msk"

val_input_it= "/home/revientaelp/Documentos/Bases de datos/Kika/300/val/org"
val_target_it= "/home/revientaelp/Documentos/Bases de datos/Kika/300/val/msk"

#Listas de las imagenes
img_size = (304, 304)
num_classes = 2
batch_size_t = 1
batch_size_v = 1

input_img_paths = sorted(
    [
        os.path.join(input_dir, fname)
        for fname in os.listdir(input_dir) 
        if fname.endswith(".png")
    ]
)

target_img_paths = sorted(
    [
        os.path.join(target_dir, fname)
        for fname in os.listdir(target_dir)
        if fname.endswith(".png") and not fname.startswith(".")
    ]
)

val_input_paths = sorted(
    [
        os.path.join(val_input_it, fname)
        for fname in os.listdir(val_input_it)
        if fname.endswith(".png") and not fname.startswith(".")
    ]
)

val_target_paths = sorted(
    [
        os.path.join(val_target_it, fname)
        for fname in os.listdir(val_target_it)
        if fname.endswith(".png") and not fname.startswith(".")
    ]
)

ID_input_t= np.arange(len(input_img_paths))
ID_input_v= np.arange(len(val_input_paths))

In [4]:
#Generador de los ejemplos de entrada
class MiClasificacion(tf.keras.utils.Sequence):
    def __init__(self, input_img_paths, target_img_paths, ID_input, shuffle= True, batch_size= 32, img_size= (304, 304, 3), train= True):

        self.batch_size = batch_size
        self.img_size = img_size
        self.input_img_paths = input_img_paths
        self.target_img_paths = target_img_paths
        self.ID_input = ID_input
        self.shuffle= shuffle
        self.cont= 0
        self.train= train

    def __len__(self):
        # Calcula el numero de pasos por epoca.
        return len(self.target_img_paths) // self.batch_size

    def __getitem__(self, idx):
        """Returns tuple (input, target) correspond to batch #idx."""

        # Combinamos los datos en cada epoca, pero solo para los datos de entrenamiento
        if (self.cont== len(self.target_img_paths) // self.batch_size) or (self.cont== 0) and (self.train== True):
            np.random.shuffle(self.ID_input)
            self.cont= 0
            
        i = idx * self.batch_size
        batch_img= []
        batch_tar= []
        if self.train== True:
            self.cont =self.cont +1

        #De acuerdo a los indices en ID_input tomamos las imagenes de los paths correspondientes
        for ig in self.ID_input[i : i + self.batch_size]:
            batch_img.append(self.input_img_paths[ig])
            batch_tar.append(self.target_img_paths[ig])

        X, Y= self.__data_generation(batch_img, batch_tar)                                          # Mandamos llamar a la funcion generadora de los ejemplos

        return X, Y

    def __data_generation(self, biip, btip):
            
        # Creacion del tensor con dimensiones las dimensiones de entrada
        x = np.zeros((self.batch_size,) + self.img_size + (3,), dtype="float32")
        #x = np.zeros((self.batch_size,) + self.img_size + (1,), dtype="float32")                    # Generamos un tensor que almacenara las imagenes importadas     
        for j, path in enumerate(biip):
            img = load_img(path, target_size=self.img_size)                                         # Cragamos las imagenes de entrada 
            x[j] = np.array(img)/255.0                                                              # Normalizamos y agregamos la imagen al tensor

        y = np.zeros((self.batch_size,) + self.img_size + (1,), dtype="uint8")                      # Generamos un tensor que almacenara las mascaras importadas 
        for j, path in enumerate(btip):
            img = load_img(path,target_size= self.img_size,color_mode="grayscale")                  # Importamos la imagne en escala de grises
            y[j] = np.expand_dims(img, 2)
            for r in range(y.shape[1]):
                for g in range(y.shape[2]):
                    if y[j, r, g, 0]!= 0:                                                           # Nos aseguramos de que las venas etiquetadas resulten se un 1
                        y[j, r, g, 0]= 1

        return x, y

In [5]:
# Generamos la secuencia del DownSampling de UNET
def conv_code(entrada, filtros, ker_reg= None):
  x= Conv2D(filtros, 3, padding= "same", activation= "relu",  kernel_regularizer= ker_reg)(entrada)
  x= BatchNormalization()(x)

  x= Conv2D(filtros, 3, padding= "same", activation= "relu", kernel_regularizer= ker_reg)(x)
  x= BatchNormalization()(x)

  residual= x
  x= MaxPooling2D(3, strides= 2, padding= "same")(x)
  
  return residual, x

# Generamos la secuencia BottleNeck correspondiente a l
def botle(entrada):
  x= Conv2D(512, 3, padding= "same", activation= "relu",  kernel_regularizer= tf.keras.regularizers.L1L2(l1=0.001, l2=0.001))(entrada)
  x= BatchNormalization()(x)
  x= Conv2D(512, 3, padding= "same", activation= "relu",  kernel_regularizer= tf.keras.regularizers.L1L2(l1=0.001, l2=0.001))(x)
  x= BatchNormalization()(x)

  x= Conv2DTranspose(256, 3,padding= "same" )(x)

  return x

# Generamos el UPSAMPLING 
def conv_decode(entrada, filtros, ker_reg= None):
  x= Conv2D(filtros, 3, padding= "same", activation= "relu", kernel_regularizer= ker_reg)(entrada)
  x= BatchNormalization()(x)
  x= Conv2D(filtros, 3, padding= "same", activation= "relu", kernel_regularizer= ker_reg)(x)
  x= BatchNormalization()(x)

  x= Conv2DTranspose(int(filtros/2), 3,padding= "same" )(x)

  return x

# Para ir subiendo entre capa y capa
def crop_and_cat(entrada, res):
  x= UpSampling2D(2)(entrada)
  
  return concatenate([x, res])

#La capa final de salida
def capa_final(entrada, filtros, num_clases):
  x= Conv2D(filtros, 3, padding= "same", activation= "relu")(entrada)
  x= BatchNormalization()(x)

  x= Conv2D(filtros, 3, padding= "same", activation= "relu")(x)
  x= BatchNormalization()(x)

  x= Conv2D(num_clases, 3, padding= "same", activation= "softmax")(x)

  return x

In [9]:
#Generamos el modelo
def get_model(img_size, num_classes):
  inputs= tf.keras.Input(shape= img_size+ (3,))
  
  res_1, conv_1= conv_code(inputs, 64)
  res_2, conv_2= conv_code(conv_1, 128)
  res_3, conv_3= conv_code(conv_2, 256, tf.keras.regularizers.L1L2(l1=0.001, l2=0.001))

  bot= botle(conv_3)

  crop1= crop_and_cat(bot, res_3)
  decode_3= conv_decode(crop1, 256, tf.keras.regularizers.L1L2(l1=0.001, l2=0.001))

  crop2= crop_and_cat(decode_3, res_2)
  decode_2= conv_decode(crop2, 128)

  crop3= crop_and_cat(decode_2, res_1)
  x= Dropout(rate= 0.5)(crop3)
  outputs= capa_final(x, 64, 2)

  model= tf.keras.Model(inputs, outputs)
  return model

def entrenamient_modelo(train_gen, val_gen, DIR_save, epocas= 30, visualizacion= False):

    #Generamos los callback necesarios
    checkpoint= ModelCheckpoint(DIR_save+ "03092021_S1.hdf5", monitor= 'val_loss', save_best_only= True, mode= 'min', save_weights_only= False)
    reduceLROnPlat= ReduceLROnPlateau(monitor= 'val_loss', factor= 0.8, patience= 3, min_delta= 0.001, cooldown= 7, min_lr= 0.0001)
    #tensorboard_callback = TensorBoard(log_dir= datetime.now().strftime("%Y%m%d-%H%M%S"))
    callbacks= [EarlyStopping(patience= 12,  monitor='val_loss'), History(), checkpoint, reduceLROnPlat]#, tensorboard_callback]
    metrica= [tf.keras.metrics.CategoricalAccuracy()]

    #Compilamos con la funcion perdida y el optimizador que utilizaremos
    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate= 0.001), loss= "sparse_categorical_crossentropy", metrics= metrica) #loss= "binary_crossentropy", metrics= ['accuracy'])
    history= model.fit(train_gen, epochs=epocas, validation_data=val_gen, callbacks=callbacks)
  
def infor_model(img_size):
  model = get_model(img_size, 2)                                   # Generamos el modelo con el tamaño de las imagenes de entrada y la salida esperada
  model.summary()  

def carga_mod(DIR):
    return tf.keras.models.load_model(DIR)

In [10]:
#Creamos los generadoras para el entrenamiento y la validación
train_gen = MiClasificacion(
    input_img_paths, target_img_paths, ID_input_t, shuffle= True, batch_size= batch_size_t, img_size= img_size, train= True
)

val_gen = MiClasificacion(
    val_input_paths, val_target_paths, ID_input_v, shuffle= True, batch_size=batch_size_v, img_size= img_size, train= False
)

#infor_model((304, 304))

In [11]:
# Comenzamos el entrenamiento
#DIR= 'E:/rortiz324/Proyecto/Bases_de_datos/ModelosGuardados/Grises/SGD/140521/14052021_400.hdf5'
#DIR= 'E:/rortiz324/Proyecto/Bases_de_datos/ModelosGuardados/Filtros/SGD/120521/12052021_400.hdf5'
#DIR= 'E:/rortiz324/Proyecto/Bases_de_datos/ModelosGuardados/Normales/SGD/140521/14052021_400.hdf5'
#DIR= 'E:/rortiz324/Proyecto/Bases_de_datos/ModelosGuardados/Comb/SGD/150521/15052021_400.hdf5'

#DIR= 'E:/rortiz324/Proyecto/Bases_de_datos/ModelosGuardados/Filtros/ADAM/240421/24042021_400_2.hdf5'
#DIR= 'E:/rortiz324/Proyecto/Bases_de_datos/ModelosGuardados/Normales/ADAM/040521_1/04052021_400.hdf5'
#DIR= 'E:/rortiz324/Proyecto/Bases_de_datos/ModelosGuardados/Comb/ADAM/080521/08052021_400.hdf5'
#DIR= 'E:/rortiz324/Proyecto/Bases_de_datos/ModelosGuardados/Grises/ADAM/300421/29042021_400_1.hdf5'

#model= carga_mod(DIR)
model= get_model(img_size, 2)

DIR_save_best= '/home/revientaelp/Documentos/Modelos/'
entrenamient_modelo(train_gen, val_gen, DIR_save_best, 400, False)

Epoch 1/400
17833/17833 [==============================] - 2118s 119ms/step - loss: 2.2359 - categorical_accuracy: 0.8959 - val_loss: 2.4010 - val_categorical_accuracy: 0.8324 - lr: 0.0010
Epoch 2/400
17833/17833 [==============================] - 2108s 118ms/step - loss: 1.7561 - categorical_accuracy: 0.8871 - val_loss: 1.5997 - val_categorical_accuracy: 0.8008 - lr: 0.0010
Epoch 3/400
17833/17833 [==============================] - 2054s 115ms/step - loss: 1.6418 - categorical_accuracy: 0.8841 - val_loss: 1.8765 - val_categorical_accuracy: 0.7660 - lr: 0.0010
Epoch 4/400
17833/17833 [==============================] - 2056s 115ms/step - loss: 1.2727 - categorical_accuracy: 0.8819 - val_loss: 1.1335 - val_categorical_accuracy: 0.8637 - lr: 0.0010
Epoch 5/400
17833/17833 [==============================] - 2054s 115ms/step - loss: 0.9363 - categorical_accuracy: 0.8804 - val_loss: 1.1700 - val_categorical_accuracy: 0.8289 - lr: 0.0010
Epoch 6/400
17833/17833 [==============================

KeyboardInterrupt: 

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np

#from IPython.display import Image, display
from tensorflow.keras.preprocessing.image import load_img
import PIL
from PIL import ImageOps
m = tf.keras.metrics.AUC()


val_input_it= "E:/rortiz324/Proyecto/Bases_de_datos/Imagenes_Grises/Validacion/Img"
val_target_it= "E:/rortiz324/Proyecto/Bases_de_datos/Imagenes_Grises/Validacion/Mask"

#val_input_it= "E:/rortiz324/Proyecto/Bases_de_datos/Imagenes_Mejoradas/Validacion_Total/Img"
#val_target_it= "E:/rortiz324/Proyecto/Bases_de_datos/Imagenes_Mejoradas/Validacion_Total/Mask"

#val_input_it= "E:/rortiz324/Proyecto/Bases_de_datos/Imagenes_Normales/Validacion_Total/Img"
#val_target_it= "E:/rortiz324/Proyecto/Bases_de_datos/Imagenes_Normales/Validacion_Total/Mask"

#val_input_it= "E:/rortiz324/Proyecto/Bases_de_datos/Imagenes_Combinadas/Validacion_T/Img"
#val_target_it= "E:/rortiz324/Proyecto/Bases_de_datos/Imagenes_Combinadas/Validacion_T/Mask"

ret= 0

val_input_paths = sorted(
    [
        os.path.join(val_input_it, fname)
        for fname in os.listdir(val_input_it)
        if fname.endswith(".png") and not fname.startswith(".")
    ]
)

val_target_paths = sorted(
    [
        os.path.join(val_target_it, fname)
        for fname in os.listdir(val_target_it)
        if fname.endswith(".png") and not fname.startswith(".")
    ]
)


for l in range(10):
    img_1= load_img(val_input_paths[l], target_size=(400, 400, 3))
    img_1= (np.array(img_1).reshape(-1,400,400,3))/255.0
    val_preds = model.predict(img_1)
    mask = np.argmax(val_preds[0], axis=-1)
    mask = np.expand_dims(mask, axis=-1)

    im = load_img(val_target_paths[l], target_size= (400, 400), color_mode="grayscale")  
    y= np.array(im).reshape(400,400,1)
    for r in range(y.shape[0]):
        for g in range(y.shape[1]):
            if y[r, g, 0]!= 0:
                y[r, g, 0]= 1

    m.update_state(y, mask)
    ret= ret + m.result().numpy()

ret= ret/10
print(ret)
    

#img = PIL.ImageOps.autocontrast(tf.keras.preprocessing.image.array_to_img(mask))
#plt.imshow(img, cmap="gray")
#plt.show()
#plt.imsave("SGD_Norm.png", img, cmap="gray")
#display(img)

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np

#from IPython.display import Image, display
from tensorflow.keras.preprocessing.image import load_img
import PIL
from PIL import ImageOps

val_input_it= "E:/rortiz324/Proyecto/Bases_de_datos/Imagenes_Grises/Validacion/Img/0.png"

img_1= load_img(val_input_it, target_size=(400, 400, 3))
img_1= (np.array(img_1).reshape(-1,400,400,3))/255.0
val_preds = model.predict(img_1)
mask = np.argmax(val_preds[0], axis=-1)
mask = np.expand_dims(mask, axis=-1)

img = PIL.ImageOps.autocontrast(tf.keras.preprocessing.image.array_to_img(mask))
plt.imshow(img, cmap="gray")
plt.show()
plt.imsave("SGD_Comb.png", img, cmap="gray")